# Creación de Dataset de Municipios de Colombia

#### Tarea #1: Obtener la información para un municipio

In [1]:
# Importar librerías principales

import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import numpy as np
import pickle

In [3]:
# Cargar el contenido de la pagina web (Recurso: Wikipedia)
w = requests.get("https://es.wikipedia.org/wiki/Bogot%C3%A1")

# Convertir a un objeto "BeautifulSoup"
bsobj = bs(w.content)

# Imprimir (Revisar estructura del código html)
print(bsobj.prettify())

<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="es">
 <head>
  <meta charset="utf-8"/>
  <title>
   Bogotá - Wikipedia, la enciclopedia libre
  </title>
  <script>
   document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":[",\t."," \t,"],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","enero","febrero","marzo","abril","mayo","junio","julio","agosto","septiembre","octubre","noviembre","diciembre"],"wgRequestId":"eb5ee6e5-ea46-4f19-81b1-3d2d8652a5d8","wgCSPNonce":!1,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"Bogotá","wgTitle":"Bogotá","wgCurRevisionId":137593937,"wgRevisionId":137593937,"wgArticleId":5131,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Wikipedia:Páginas que utilizan Timeline","Wikipedia:Páginas con referencias con parámetros obsoletos","Wikipedia:Artículos con enlaces extern

In [4]:
# Revisar la información de la "tabla de datos" usando una clase específica

info_caja = bsobj.find(class_="infobox geography vcard")
info_filas = info_caja.find_all("tr")
for fila in info_filas:
    print(fila.prettify())

<tr>
 <th class="cabecera mapa fn org" colspan="3" style="text-align:center;background-color:transparent;color:black;padding:0;">
  Bogotá
 </th>
</tr>

<tr>
 <td colspan="3" style="text-align:center;font-size:100%; font-weight:bold; background-color:#cddeff;">
  <a href="/wiki/Capital_(pol%C3%ADtica)" title="Capital (política)">
   Capital
  </a>
  de
  <a href="/wiki/Colombia" title="Colombia">
   Colombia
  </a>
 </td>
</tr>

<tr>
 <td colspan="3" style="text-align:center;">
  <table align="right" bgcolor="white" cellspacing="0" style="border:1px solid white">
   <tbody>
    <tr>
     <td style="padding:2px 0px 0px 2px">
      <table bgcolor="white" cellspacing="0" style="border:0px">
       <tbody>
        <tr>
         <td style="padding:0px 2px 2px 0px">
          <a class="image" href="/wiki/Archivo:Centro_internacional.JPG" title="Centro">
           <img alt="Centro" data-file-height="1080" data-file-width="1920" decoding="async" height="152" src="//upload.wikimedia.org/wikipe


<tr>
 <th scope="row" style="text-align:left;border:0;padding:1px 7px;font-weight:100;">
  • Media
 </th>
 <td colspan="2" style="border:0;padding:1px 7px 1px 1px;">
  2640
  <a class="mw-redirect" href="/wiki/Msnm" title="Msnm">
   m s. n. m.
  </a>
 </td>
</tr>

<tr>
 <th scope="row" style="text-align:left;border:0;padding:1px 7px;font-weight:100;">
  • Máxima
 </th>
 <td colspan="2" style="border:0;padding:1px 7px 1px 1px;">
  4000
  <sup class="reference separada" id="cite_ref-2">
   <a href="#cite_note-2">
    <span class="corchete-llamada">
     [
    </span>
    2
    <span class="corchete-llamada">
     ]
    </span>
   </a>
  </sup>
  ​ m s. n. m.
 </td>
</tr>

<tr>
 <th scope="row" style="text-align:left;border:0;padding:1px 7px;font-weight:100;">
  • Mínima
 </th>
 <td colspan="2" style="border:0;padding:1px 7px 1px 1px;">
  2540
  <sup>
   [
   <i>
    <a href="/wiki/Wikipedia:Verificabilidad" title="Wikipedia:Verificabilidad">
     cita requerida
    </a>
   </i>
   ]
  <

In [5]:
# Función para obtener los valores dados diferentes casos, además, se reemplazan caracteres extraños (unicode).
def obtener_valores(fila_data):
    if fila_data.find("div") and fila_data.find("br"):
        return [texto for texto in fila_data.stripped_strings]
    else:
        return fila_data.get_text(" ").replace("\xa0", " ").replace("\u200b", "").strip()

# Crear función para eliminar etiquetas [1] (span) solo en el caso de que no tengan información relevante
def limpiar_etiquetas(bs_objeto):
    if bs_objeto.find_all("sup", {'class': 'reference separada'}):
        for etiqueta in bs_objeto.find_all("sup"):
            etiqueta.decompose()
    pass

limpiar_etiquetas(bsobj)
         
municipios_info = {}

# Crear una lista de títulos para tomar el indice desde el cuál empezar a obtener los datos
titulos = [fila.get('title') for fila in info_caja.find_all("a") if fila.get('title') is not None]
contador = 0
for indice, fila in enumerate(info_filas):
    if indice == 0 and fila.find("th"):
        municipios_info['municipio'] = fila.find("th").get_text(" ", strip=True)
    # En caso de que el tag "th" no contenga información o no exista, tomar el nombre del municipio del encabezado (tag "h1")
    elif indice == 0 and not fila.find("th"):
        municipios_info['municipio'] = bsobj.find("h1").get_text(" ", strip=True)
    elif indice < titulos.index('Coordenadas geográficas'): # Imágenes y otros datos irrelevantes no serán tenidos en cuenta
        continue
    else: # Las filas de tabla (tr) que no contengan elementos "th" & "td" serán evitadas
        try:
            # Se obtiene las llaves y se reemplazan caracteres extraños. 
            # Se anexa un número a aquellas llaves que posean el mismo nombre "* Total" para evitar errores en los datos
            if fila.find("th").get_text(" ", strip=True) == "• Total":
                llaves = fila.find("th").get_text(" ", strip=True).replace("•", "").strip() + f"_{contador}"
                valores = obtener_valores(fila.find("td"))
                municipios_info[llaves] = valores
                contador += 1
            else:
                llaves = fila.find("th").get_text(" ", strip=True).replace("•", "").replace("\u200b", "").strip()
                valores = obtener_valores(fila.find("td"))
                municipios_info[llaves] = valores
        except AttributeError as e:
            pass

municipios_info

{'municipio': 'Bogotá',
 'Subdivisiones': '20  localidades 1922  barrios',
 'Localidades': ['Ver lista',
  'Usaquén',
  'Chapinero',
  'Santa Fe',
  'San Cristóbal',
  'Usme',
  'Tunjuelito',
  'Bosa',
  'Kennedy',
  'Fontibón',
  'Engativá',
  'Suba',
  'Barrios Unidos',
  'Teusaquillo',
  'Los Mártires',
  'Antonio Nariño',
  'Puente Aranda',
  'La Candelaria',
  '(Centro Histórico)',
  'Rafael Uribe Uribe',
  'Ciudad Bolívar',
  'Sumapaz'],
 'Eventos históricos': '',
 'Fundación': '6 de agosto de 1538 (483 años)   ( Gonzalo Jiménez de Quesada )',
 'Erección': '3 de diciembre de 1548 (472 años)',
 'Creación': '17 de diciembre de 1954 (66 años) (conformación del  Distrito Capital )',
 'Superficie': '',
 'Total_0': '1775  km²',
 'Altitud': '',
 'Media': '2640  m s. n. m.',
 'Máxima': '4000  m s. n. m.',
 'Mínima': '2540  m s. n. m.',
 'Clima': 'Oceánico mediterráneo  Csb',
 'Población (2020)': '',
 'Total_1': '7 743 955 hab.',
 'Densidad': '4907,45 hab/km²',
 'Urbana': '7 412 566 hab.'

#### Tarea #2: Obtener la información para todos los municipios de Colombia

In [24]:
# Cargar el contenido de la página (Lista de municipios de Colombia por población)
url = requests.get("https://es.wikipedia.org/wiki/Anexo:Municipios_de_Colombia_por_poblaci%C3%B3n")

# Convertir a un objeto BS
bso = bs(url.content)

# Imprimir HTML
print(bso.prettify())

In [25]:
# Extraer las etiquetas "href" para la clase especificada
municipios = bso.select(".wikitable.sortable td")
rutas = [m.a['href'] for m in municipios if m.a and m.get('align') is None]
print(rutas)

In [26]:
# Extraer las rutas de cada municipio y formar junto al dominio, los enlaces completos.

dominio = "https://es.wikipedia.org"
enlaces = []
for ruta in rutas:
    enlaces.append(dominio + ruta)

In [27]:
# En algunos casos, las rutas estaban conectadas a municipalidades de otros paises. Se cambian por los enlaces correctos
for n, i in enumerate(enlaces):
    if i == 'https://es.wikipedia.org/wiki/Aquitania':
        enlaces[n] = 'https://es.wikipedia.org/wiki/Aquitania_(Boyac%C3%A1)'
    elif i == 'https://es.wikipedia.org/wiki/Saman%C3%A1':
        enlaces[n] = 'https://es.wikipedia.org/wiki/Saman%C3%A1_(Caldas)'
    elif i == 'https://es.wikipedia.org/wiki/Roncesvalles':
        enlaces[n] = 'https://es.wikipedia.org/wiki/Roncesvalles_(Tolima)'
              

In [28]:
# Crear función para obtener los valores dependiendo de la etiqueta, además, se eliminan caracteres extraños.
def obtener_valores(fila_data):
    if fila_data.find("div") and fila_data.find("br"):
        return [texto for texto in fila_data.stripped_strings]
    else:
        return fila_data.get_text(" ").replace("\xa0", " ").replace("\u200b", "").strip()

# Crear función para eliminar etiquetas [1] (span) solo en el caso de que no tengan información relevante
def limpiar_etiquetas(bs_objeto):
    if bs_objeto.find_all("sup", {'class': 'reference separada'}):
        for etiqueta in bs_objeto.find_all("sup"):
            etiqueta.decompose()
    pass

# Crear función para obtener la información de un municipio dada su dirección url.
def obtener_info(url):
    
    w = requests.get(url)
    
    bsobj = bs(w.content)
    info_caja = bsobj.find(class_="infobox geography vcard")
    info_filas = info_caja.find_all("tr")
    
    limpiar_etiquetas(bsobj)
    
    municipios_info = {}

    # Crear una lista de títulos para tomar el indice desde el cuál empezar a obtener los datos
    titulos = [fila.get('title') for fila in info_caja.find_all("a") if fila.get('title') is not None]
    contador = 0
    for indice, fila in enumerate(info_filas):
        if indice == 0 and fila.find("th"):
            municipios_info['municipio'] = fila.find("th").get_text(" ", strip=True)
        # En el caso de que no exista la etiqueta "th", será necesario tomar el nombre de la cabecera "h1" del código HTML.
        elif indice == 0 and not fila.find("th"):
            municipios_info['municipio'] = bsobj.find("h1").get_text(" ", strip=True)
        elif indice < titulos.index('Coordenadas geográficas'): # Imágenes y otros datos irrelevantes no serán tenidos en cuenta
            continue
        # Manejo de excepciones y reemplazo de strings en la obtención de los datos
        else:
            try:# Caso especial: En el caso de que una dos etiquetas "th" contengan el mismo texto, se le agrega un caracter 
                # adicional al texto para diferenciarlos, y así evitar errores en la conexión con los "td" relacionados.
                if fila.find("th").get_text(" ", strip=True) == "• Total":
                    llaves = fila.find("th").get_text(" ", strip=True).replace("•", "").strip() + f"_{contador}"
                    valores = obtener_valores(fila.find("td"))
                    municipios_info[llaves] = valores
                    contador += 1
                # Evitar tomar los datos de IDH y Población Metropolitana, debido a que solo se encuentran contabilizados en pocos municipios (Ciudades Capitales)
                elif 'IDH' in fila.find("th").get_text(" ", strip=True).upper() or 'METROPOLITANA' in fila.find("th").get_text(" ", strip=True).upper():
                    continue
                else:
                    llaves = fila.find("th").get_text(" ", strip=True).replace("•", "").replace("\u200b", "").strip()
                    valores = obtener_valores(fila.find("td"))
                    municipios_info[llaves] = valores
            except AttributeError as e:
                pass

    return municipios_info

In [11]:
# Probar la función "obtener_info()" para solo un municipio. Verificar que funcione correctamente.
obtener_info("https://es.wikipedia.org/wiki/Medell%C3%ADn")

{'municipio': 'Medellín',
 'Coordenadas': '6°14′41″N  75°34′29″O \ufeff / \ufeff 6.2447472222222,  -75.574827777778 Coordenadas :  6°14′41″N  75°34′29″O \ufeff / \ufeff 6.2447472222222,  -75.574827777778',
 'Entidad': 'Municipio',
 'País': 'Colombia',
 'Departamento': 'Antioquia',
 'Región': 'Valle de Aburrá',
 'Alcalde': 'Daniel Quintero Calle  ( Independientes )   2020 - 2023',
 'Subdivisiones': '5  corregimientos  16  comunas  275  barrios',
 'Corregimientos': ['Ver lista',
  'Altavista',
  'Palmitas',
  'San Antonio de Prado',
  'San Cristóbal',
  'Santa Elena'],
 'Comunas': ['Ver lista',
  'Aranjuez',
  'Belén',
  'Buenos Aires',
  'Castilla',
  'Doce de Octubre',
  'El Poblado',
  'Guayabal',
  'La América',
  'La Candelaria',
  'Laureles - Estadio',
  'Manrique',
  'Popular',
  'Robledo',
  'San Javier',
  'Santa Cruz',
  'Villa Hermosa'],
 'Eventos históricos': '',
 'Fundación': '2 de marzo  de  1616  (405 años)  ( Miguel de Aguinaga y Mendigoitia )',
 'Erección': '21 de agosto

In [29]:
# Se inicia el proceso de extracción de datos para todos los municipios (Guardar todos los diccionarios en una lista)

municipios = []
for enlace in enlaces:
    try:
        municipios.append(obtener_info(enlace))
    except Exception as e:
        print(enlace)
        print(e)    

https://es.wikipedia.org/wiki/Isla_de_San_Andr%C3%A9s_(Colombia)
'NoneType' object has no attribute 'find_all'
https://es.wikipedia.org/wiki/Cerrito_(Santander)
'NoneType' object has no attribute 'find_all'
https://es.wikipedia.org/wiki/Isla_de_Providencia
'NoneType' object has no attribute 'find_all'
https://es.wikipedia.org/wiki/%C3%9Atica
'NoneType' object has no attribute 'find_all'


In [30]:
len(municipios)
# Se logra obtener los datos de 1118 municipios de 1122 existentes. 
# Para el caso de los municipios de "Isla de San Andrés" e "Isla de Providencia" las tablas no soportan el método "find_all"
# En los otros dos casos, no existe una tabla para la obtención de los datos.

1118

In [31]:
# Definir una función para guardar el archivo .json y otra para cargar el archivo .json

import json

def guardar_datos(titulo, datos):
    with open(titulo, 'w', encoding='utf-8') as a:
        json.dump(datos, a, ensure_ascii=False, indent=2)
        
def cargar_datos(titulo):
    with open(titulo, encoding="utf-8") as a:
        return json.load(a)

In [32]:
# Guardar resultados obtenidos hasta el momento.

guardar_datos("municipios_col.json", municipios)

#### Tarea #3: Limpieza de Datos

In [16]:
# Cargar datos

municipios_col = cargar_datos("municipios_col.json")

In [17]:
# Imprimir todas las distintas llaves de la lista de dictionarios

all_keys = set().union(*(d.keys() for d in municipios_col))
all_keys

# Se identifican diversas llaves que no contienen información relevante, otras que contienen valores faltantes excesivos,
# algunas difieren en el nombre pero guardan valores similares.

{'',
 'Aeropuerto',
 'Alcald',
 'Alcalde',
 'Alcalde 2020 - 2023 Camilo Correa Álvarez',
 'Alcalde 2020-2023',
 'Alcalde Distrital',
 'Alcalde Federico alfonso  gomez jimenez',
 'Alcalde Municipal',
 'Alcalde de Plato',
 'Alcalde, Alcalde (Interino)',
 'Alcaldes',
 'Alcaldesa',
 'Altitud',
 "Autoridad tradicional Ne'jwesx 2019 - 2021",
 'Ave',
 'Barrios',
 'Cabecera',
 'Cabecera municipal',
 'Capital',
 'Caseríos',
 'Centros Poblados',
 'Clima',
 'Comunas',
 'Coordenadas',
 'Corregidor',
 'Corregimientos',
 'Creación',
 'Curso de agua',
 'Código Dane (Divipola)',
 'Código Dian',
 'Código ZIP',
 'Código postal',
 'Densidad',
 'Departamento',
 'Departamento Boyacá',
 'Departamentos',
 'Departamentos de abril',
 'Entidad',
 'Erección',
 'Eventos históricos',
 'Fiestas mayores',
 'Flor',
 'Fundación',
 'Gentilicio',
 'Hermanada con',
 'Huso horario',
 'Idioma oficial',
 'Localidades',
 'Matrícula',
 'Media',
 'Moneda',
 'Municipio',
 'Municipio Barrancominas',
 'Máxima',
 'Mínima',
 'Nombr

In [144]:
# Para este ejercicio, se trabaja con la información bajo un DataFrame (para facilitar algunas operaciones de limpieza)

municipios_df = pd.DataFrame(municipios_col)

In [73]:
# Aquitania Boyacá (Cambiar link) https://es.wikipedia.org/wiki/Aquitania_(Boyac%C3%A1)
# Samaná Caldas (Cambiar link) https://es.wikipedia.org/wiki/Saman%C3%A1_(Caldas)
# Roncesvalles Tolima (Cambiar link) https://es.wikipedia.org/wiki/Roncesvalles_(Tolima)
# Eliminar key "Aeropuerto"
# Eliminar key "Autoridad tradicional Ne'jwesx 2019 - 2021"
# Eliminar "Flor", "Arbol", "Ave", "Cabecera", "Cabecera municipal", "Capital", "Barrios", "Veredas", "Caseríos"
# Eliminar "Centros Poblados", "Hermanada con", "Idioma oficial", "Moneda", "Máxima", "Mínima", "Matrícula"
# Unir Alcalde/Corregidor
# Unir localidades, comunas, barrios
# Eliminar "Otras S(s)uperficies" 'PIB (nominal)','PIB per cápita','Partidos gobernantes'
# Eliminar 'Presidente regional','Presupuesto anual', 'Temperatura', 'Temperatura promedio', 'Total_2'
# Ajustar verdadero nombre '[[Departamentos de \nColombia|Departamento]]','[xau[Subregiones de Antioquia'


In [145]:
municipios_df.head()

,municipio,Subdivisiones,Localidades,Eventos históricos,Fundación,Erección,Creación,Superficie,Total_0,Altitud,...,Barrios,Población (2015\nclima =12ºC),Población ( 2018 ),Alcald,Departamento Boyacá,Población (4512),departamento,Corregidor,Departamentos de abril,Municipio Barrancominas
0,Bogotá,20 localidades 1922 barrios,"[Ver lista, Usaquén, Chapinero, Santa Fe, San ...",,6 de agosto de 1538 (483 años) ( Gonzalo Jim...,3 de diciembre de 1548 (472 años),17 de diciembre de 1954 (66 años) (conformació...,,1775 km²,,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Medellín,5 corregimientos 16 comunas 275 barrios,NaN,,2 de marzo de 1616 (405 años) ( Miguel de ...,21 de agosto de 1813,NaN,,382 km²,,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Cali Santiago de Cali,15 corregimientos 22 comunas 249 barrios,NaN,,25 de julio de 1536 (485 años) (por Sebas...,17 de junio de 1559,NaN,,564 km²,,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Barranquilla,2 corregimientos 5 localidades 188 barrios,"[Ver lista, Riomar, Norte-Centro Histórico, Su...",,Entre 1627 y 1637,7 de abril de 1813,NaN,,154 km²,,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Cartagena de Indias,NaN,NaN,,1 de junio de 1533 (488 años) ( Pedro de H...,1533,NaN,,609.1 km²,,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


##### ** Eliminar columnas con información irrelevante, incompleta o nula **

In [146]:

columnas_erradas = ["Aeropuerto", "Flor", "Arbol", "Árbol", "Ave", "Cabecera", "Cabecera municipal",
        "Capital", "Barrios", "Veredas", "Caseríos", "Centros Poblados", "Nombre", "Fiestas mayores",
        "Hermanada con","Idioma oficial","Moneda","Máxima","Mínima","Matrícula","Patrono(a)","Localidades",
        "Otras Superficies:","PIB (nominal)","PIB per cápita","Partidos gobernantes","Corregimientos",
        "Otras superficies:", "Presidente regional","Presupuesto anual", "Temperatura", "Creación",
        "Temperatura promedio", "Total_2", "Eventos históricos","Superficie","Altitud", "Comunas",
        "Curso de agua","Código Dane (Divipola)","Código Dian","Correspondencia actual","Subdivisiones",
        "Autoridad tradicional Ne'jwesx 2019 - 2021", "Otros idiomas", "", "Prefijo telefónico",
        "Región", "Provincia", "Zona"]

for col in columnas_erradas:
    if col in municipios_df.columns:
        municipios_df = municipios_df.drop(col, 1)
    continue

#municipio['Superficie'] = municipio.pop('Total_0')
#municipio['Altitud Media'] = municipio.pop('Media')
# Unir Región, Subregión y Zona

In [147]:
# Las columnas 'Total_0', Total_1', 'Urbana' y 'Media' contiene la información correspondiente 
# a 'Superficie', 'Población', 'Pobl. Urbana' y 'Altitud Media' respectivamente, por lo tanto,
# cambiamos el nombre de las columnas.

municipios_df = municipios_df.rename(columns={'Total_0': 'Superficie', 'Total_1': 'Población Total',
                                              'Media': 'Altitud Media', 'Urbana': 'Pobl. Urbana'})

##### ** Unir columnas con igual información pero diferente nombre **

In [149]:
# Reducir a una sola columna aquellas que contienen la misma información pero difieren en su encabezado

from re import search

columnas = {'lcald': 'Alcalde', 'epartamento': 'Departamento', 'unicipio': 'Municipio',
           'oblación': 'Población Total', 'Subregi': 'Subregión', 'Código': 'Código postal'}

for col in municipios_df.columns:
    for key, value in columnas.items():
        if search(key, col) and col != value:
            municipios_df[value].fillna(municipios_df[col], inplace=True)
            municipios_df = municipios_df.drop(col, 1)
        continue

municipios_df = municipios_df.drop("Subregión", 1)

In [151]:
# Unir las columnas Alcalde/Corregidor

municipios_df['Alcalde'].fillna(municipios_df['Corregidor'], inplace=True)
municipios_df = municipios_df.drop('Corregidor', 1)

##### Extraer y añadir la información de códigos postales para todos los municipios

In [153]:
# Extraer el contenido de la página y crear el objeto BS (Departamentos)

url = requests.get("https://codigo-postal.co/colombia/")
soup = bs(url.content)
print(soup.prettify())

<!DOCTYPE html>
<html lang="es">
 <head>
  <meta charset="utf-8"/>
  <meta content="IE=edge" http-equiv="X-UA-Compatible"/>
  <meta content="width=device-width, initial-scale=1" name="viewport"/>
  <title>
   Código Postal de Colombia
  </title>
  <meta content="codigo-postal.co" name="author"/>
  <meta content="Descubre el Código Postal de Colombia. Selecciona el departamento, luego el municipio y, por fin, filtra por la localidad o barrio para encontrar el código postal correspondiente." name="description"/>
  <meta content="286861161921582" property="fb:app_id"/>
  <meta content="Código Postal de Colombia" property="og:title"/>
  <meta content="https://codigo-postal.co/colombia/" property="og:url"/>
  <meta content="website" property="og:type"/>
  <meta content="https://cdn.codigo-postal.co/data/co/images/colombia.svg" property="og:image"/>
  <meta content="Descubre el Código Postal de Colombia. Selecciona el departamento, luego el municipio y, por fin, filtra por la localidad o bar

In [170]:
# Identificar todas las etiquetas "li" dentro de contenido de la clase especificada

info_box = soup.find(class_="column-list")
info_rows = info_box.find_all("li")
for row in info_rows:
    print(row.prettify())

<li>
 <a href="https://codigo-postal.co/colombia/amazonas/" title="Código Postal Amazonas">
  <span class="co co-amazonas">
  </span>
  Amazonas
 </a>
</li>

<li>
 <a href="https://codigo-postal.co/colombia/antioquia/" title="Código Postal Antioquia">
  <span class="co co-antioquia">
  </span>
  Antioquia
 </a>
</li>

<li>
 <a href="https://codigo-postal.co/colombia/arauca/" title="Código Postal Arauca">
  <span class="co co-arauca">
  </span>
  Arauca
 </a>
</li>

<li>
 <a href="https://codigo-postal.co/colombia/archipielago-de-san-andres/" title="Código Postal San Andres">
  <span class="co co-archipielago-de-san-andres">
  </span>
  Archipielago de San Andres
 </a>
</li>

<li>
 <a href="https://codigo-postal.co/colombia/atlantico/" title="Código Postal Atlántico">
  <span class="co co-atlantico">
  </span>
  Atlantico
 </a>
</li>

<li>
 <a href="https://codigo-postal.co/colombia/bogota-dc/" title="Código Postal Bogotá">
  <span class="co co-bogota-dc">
  </span>
  Bogota DC
 </a>
</

In [155]:
# Extraer y guardar los enlaces de cada departamento

dptos = []

for row in info_rows:
    dptos.append(row.a['href'])

dptos

['https://codigo-postal.co/colombia/amazonas/',
 'https://codigo-postal.co/colombia/antioquia/',
 'https://codigo-postal.co/colombia/arauca/',
 'https://codigo-postal.co/colombia/archipielago-de-san-andres/',
 'https://codigo-postal.co/colombia/atlantico/',
 'https://codigo-postal.co/colombia/bogota-dc/',
 'https://codigo-postal.co/colombia/bolivar/',
 'https://codigo-postal.co/colombia/boyaca/',
 'https://codigo-postal.co/colombia/caldas/',
 'https://codigo-postal.co/colombia/caqueta/',
 'https://codigo-postal.co/colombia/casanare/',
 'https://codigo-postal.co/colombia/cauca/',
 'https://codigo-postal.co/colombia/cesar/',
 'https://codigo-postal.co/colombia/choco/',
 'https://codigo-postal.co/colombia/cordoba/',
 'https://codigo-postal.co/colombia/cundinamarca/',
 'https://codigo-postal.co/colombia/guainia/',
 'https://codigo-postal.co/colombia/guaviare/',
 'https://codigo-postal.co/colombia/huila/',
 'https://codigo-postal.co/colombia/la-guajira/',
 'https://codigo-postal.co/colombia

In [194]:
# Se define la función "get_urls" la cual accede al enlace del departamento, extrae y guarda los enlaces de todos sus municipios

def get_urls(dpto, mpios):
    url = requests.get(dpto)
    soup = bs(url.content)

    info_box = soup.find(class_="col-md-8")
    info_rows = info_box.find_all("ul")

    info_lists = []
    for row in info_rows:
        info_lists.append(row.find_all("li"))
    
    for info_list in info_lists:
        for renglon in info_list:
            mpios.append(renglon.a['href'])


    return mpios

# Extraer los enlaces de todos los municipios de cada departamento. Guardar en una misma lista todos los enlaces.
mpios = []
for dpto in dptos:
    get_urls(dpto, mpios)

In [197]:
# En este caso, se procede a crear una nueva columna que contenga los nombres de cada municipio pero sin tildes (´) en caso...
# ... de que existan. Esto para facilitar el proceso de asignación del código postal para su correspondiente. Mas adelante...

import re
from unicodedata import normalize

municipios_df['New_municipios'] = ""

series = municipios_df['Municipio']
for index, value in series.items():
    municipios_df['New_municipios'][index] = normalize('NFC', re.sub(
        r"([^n\u0300-\u036f]|n(?!\u0303(?![\u0300-\u036f])))[\u0300-\u036f]+", r"\1", 
        normalize( "NFD", value), 0, re.I))

In [198]:
# Se asigna la nueva columna como indice del DataFrame
municipios_df = municipios_df.set_index('New_municipios')

In [199]:
municipios_df.head()

,Fundación,Erección,Superficie,Altitud Media,Clima,Población Total,Densidad,Pobl. Urbana,Gentilicio,Huso horario,Código postal,Coordenadas,Entidad,País,Departamento,Alcalde,Municipio
New_municipios,,,,,,,,,,,,,,,,,
Bogota,6 de agosto de 1538 (483 años) ( Gonzalo Jim...,3 de diciembre de 1548 (472 años),1775 km²,2640 m s. n. m.,Oceánico mediterráneo Csb,7 743 955 hab.,"4907,45 hab/km²",7 412 566 hab.,"Bogotano, -na",UTC-5,11,4°36′46″N 74°04′14″O,Capital de Colombia,Colombia,Cundinamarca,Claudia López (AV) (2020-2023),Bogotá
Medellin,2 de marzo de 1616 (405 años) ( Miguel de ...,21 de agosto de 1813,382 km²,1495 m s. n. m.,Tropical monzónico Am Subtropical húmedo C...,2 533 424 hab.,"6643,39 hab/km²",2 501 470 hab.,Medellinense,UTC -5,050001,"6°14′41″N 75°34′29″O ﻿ / ﻿ 6.2447472222222, ...",Municipio,Colombia,Antioquia,Daniel Quintero Calle ( Independientes ) 20...,Medellín
Cali Santiago de Cali,25 de julio de 1536 (485 años) (por Sebas...,17 de junio de 1559,564 km²,1018 m s. n. m.,Tropical seco As,2 445 281 hab.,"4382,05 hab/km²",2 408 653 hab.,"Caleño, -ña",UTC -5,NaN,NaN,NaN,NaN,NaN,NaN,Cali Santiago de Cali
Barranquilla,Entre 1627 y 1637,7 de abril de 1813,154 km²,18 m s. n. m.,Tropical seco Aw,2 274 250 hab.,"8274,35 hab/km²",2 273 646 hab.,"Barranquillero, ra",UTC-5,08000,"10°57′50″N 74°47′47″O ﻿ / ﻿ 10.963888888889, ...",Distrito,Colombia,Atlántico,Jaime Pumarejo Heins ( PCR ) ( 2020 - 2023 ),Barranquilla
Cartagena de Indias,1 de junio de 1533 (488 años) ( Pedro de H...,1533,609.1 km²,2 m s. n. m.,Semiárido cálido BSh,1 028 736 hab.,"1811,91 hab/km²",914 552 hab.,"Cartagenero, -a",UTC-5,130000,NaN,NaN,Colombia,Bolívar,William Dau Chamat ( 2020 - 2023 ),Cartagena de Indias


In [204]:
# La función "get_codes" accede al enlace del municipio, extrae y guarda el nombre del municipio y su código postal.

def get_codes(mpio, postales):
    try:
        url = requests.get(mpio)
        soup = bs(url.content)

        info_box_head = soup.find(class_="breadcrumb")
        info_rows_head = info_box_head.find_all("li")
    
        llave = info_rows_head[3].find("span").get_text(" ", strip=True)
    
        info_box_body = soup.find(class_="table-responsive")
        info_rows_body = info_box_body.find_all("tr")
    
        valor = info_rows_body[1].find("a").get_text(" ", strip=True)
    
        postales[llave] = valor

        return postales
    
    except AttributeError as e:
        print(f"¡{llave} no tiene código postal asignado!")
        pass

# Extraer el código postal para todos los municipio. Guardar la información en un diccionario.
postales = {}
for mpio in mpios:
    get_codes(mpio, postales)

¡La Victoria no tiene código ZIP asignado!
¡Antonio Nariño no tiene código ZIP asignado!
¡Candelaria no tiene código ZIP asignado!
¡Los Martires no tiene código ZIP asignado!
¡Guadalupe no tiene código ZIP asignado!


In [206]:
# La razón por la que se creó una nueva columna con los nombres de cada municipio sin tilde es porque es necesario asignarlos...
# ... usando el nombre de la llave, cuyos nombres no poseen tildes en los casos en los que debería tenerla. Por lo tanto, ...
# ... se utilizan los nuevos indices para 

municipios_df = municipios_df.drop("Código postal", 1)

municipios_df['Código Postal'] = 0
for key, value in postales.items():
    for index, val in municipios_df['Clima'].items():
        if search(key, index):
            municipios_df['Código Postal'][index] = value
            

<ipython-input-206-123bebd76470>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  municipios_df['Código Postal'][index] = value


In [210]:
# Se elimina la columna indice que fue tomada solo como referencia
municipios_df.reset_index(drop=True, inplace=True)

In [217]:
municipios_df.head()

,Fundación,Erección,Superficie,Altitud Media,Clima,Población Total,Densidad,Pobl. Urbana,Gentilicio,Huso horario,Coordenadas,Entidad,País,Departamento,Alcalde,Código Postal
Municipio,,,,,,,,,,,,,,,,
Bogotá,6 de agosto de 1538 (483 años) ( Gonzalo Jim...,3 de diciembre de 1548 (472 años),1775 km²,2640 m s. n. m.,Oceánico mediterráneo Csb,7 743 955 hab.,"4907,45 hab/km²",7 412 566 hab.,"Bogotano, -na",UTC-5,4°36′46″N 74°04′14″O,Capital de Colombia,Colombia,Cundinamarca,Claudia López (AV) (2020-2023),0
Medellín,2 de marzo de 1616 (405 años) ( Miguel de ...,21 de agosto de 1813,382 km²,1495 m s. n. m.,Tropical monzónico Am Subtropical húmedo C...,2 533 424 hab.,"6643,39 hab/km²",2 501 470 hab.,Medellinense,UTC -5,"6°14′41″N 75°34′29″O ﻿ / ﻿ 6.2447472222222, ...",Distrito,Colombia,Antioquia,Daniel Quintero Calle ( Independientes ) 20...,50035
Cali Santiago de Cali,25 de julio de 1536 (485 años) (por Sebas...,17 de junio de 1559,564 km²,1018 m s. n. m.,Tropical seco As,2 445 281 hab.,"4382,05 hab/km²",2 408 653 hab.,"Caleño, -ña",UTC -5,3°26′24″N 76°31′11″O,Distrito,NaN,Valle del Cauca,Jorge Iván Ospina,760036
Barranquilla,Entre 1627 y 1637,7 de abril de 1813,154 km²,18 m s. n. m.,Tropical seco Aw,2 274 250 hab.,"8274,35 hab/km²",2 273 646 hab.,"Barranquillero, ra",UTC-5,"10°57′50″N 74°47′47″O ﻿ / ﻿ 10.963888888889, ...",Distrito,Colombia,Atlántico,Jaime Pumarejo Heins ( PCR ) ( 2020 - 2023 ),80001
Cartagena de Indias,1 de junio de 1533 (488 años) ( Pedro de H...,1533,609.1 km²,2 m s. n. m.,Semiárido cálido BSh,1 028 736 hab.,"1811,91 hab/km²",914 552 hab.,"Cartagenero, -a",UTC-5,10°25′25″N 75°31′31″O,Distrito,Colombia,Bolívar,William Dau Chamat ( 2020 - 2023 ),130010


##### ** Convertir fechas en string a datetime **

In [370]:
# Extraer la fecha de cada string de la columna "Fundación" ("5 de junio de 1935" --> "5/junio/1935")

for idx in municipios_df.index:
    try:
        if len(municipios_df['Fundación'][idx].split()) > 4:
            municipios_df['Fundación'][idx] = ' '.join(municipios_df['Fundación'][idx].split()[:5]).replace(' de', '').replace(' ', '/')
        continue
    except Exception as e:
        print(f"{idx}: {e}")

<ipython-input-370-14e015da813a>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  municipios_df['Fundación'][idx] = ' '.join(municipios_df['Fundación'][idx].split()[:5]).replace(' de', '').replace(' ', '/')


80: 'float' object has no attribute 'split'
84: 'float' object has no attribute 'split'
89: 'float' object has no attribute 'split'
142: 'float' object has no attribute 'split'
150: 'float' object has no attribute 'split'
276: 'float' object has no attribute 'split'
293: 'float' object has no attribute 'split'
312: 'float' object has no attribute 'split'
325: 'float' object has no attribute 'split'
394: 'float' object has no attribute 'split'
403: 'float' object has no attribute 'split'
414: 'float' object has no attribute 'split'
418: 'float' object has no attribute 'split'
477: 'float' object has no attribute 'split'
488: 'float' object has no attribute 'split'
522: 'float' object has no attribute 'split'
524: 'float' object has no attribute 'split'
599: 'float' object has no attribute 'split'
635: 'float' object has no attribute 'split'
655: 'float' object has no attribute 'split'
666: 'float' object has no attribute 'split'
679: 'float' object has no attribute 'split'
735: 'float' 

In [371]:
# Extraer la fecha de cada string de la columna "Erección" ("5 de junio de 1935" --> "5/junio/1935")

for idx in municipios_df.index:
    try:
        if len(municipios_df['Erección'][idx].split()) > 4:
            municipios_df['Erección'][idx] = ' '.join(municipios_df['Erección'][idx].split()[:5]).replace(' de', '').replace(' ', '/')
        continue
    except Exception as e:
        print(f"{idx}: {e}")

<ipython-input-371-8a957b6dc445>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  municipios_df['Erección'][idx] = ' '.join(municipios_df['Erección'][idx].split()[:5]).replace(' de', '').replace(' ', '/')


13: 'float' object has no attribute 'split'
16: 'float' object has no attribute 'split'
18: 'float' object has no attribute 'split'
21: 'float' object has no attribute 'split'
31: 'float' object has no attribute 'split'
35: 'float' object has no attribute 'split'
39: 'float' object has no attribute 'split'
40: 'float' object has no attribute 'split'
41: 'float' object has no attribute 'split'
42: 'float' object has no attribute 'split'
43: 'float' object has no attribute 'split'
45: 'float' object has no attribute 'split'
56: 'float' object has no attribute 'split'
61: 'float' object has no attribute 'split'
62: 'float' object has no attribute 'split'
65: 'float' object has no attribute 'split'
77: 'float' object has no attribute 'split'
78: 'float' object has no attribute 'split'
80: 'float' object has no attribute 'split'
83: 'float' object has no attribute 'split'
86: 'float' object has no attribute 'split'
93: 'float' object has no attribute 'split'
105: 'float' object has no attri

1003: 'float' object has no attribute 'split'
1006: 'float' object has no attribute 'split'
1010: 'float' object has no attribute 'split'
1011: 'float' object has no attribute 'split'
1012: 'float' object has no attribute 'split'
1017: 'float' object has no attribute 'split'
1021: 'float' object has no attribute 'split'
1023: 'float' object has no attribute 'split'
1024: 'float' object has no attribute 'split'
1026: 'float' object has no attribute 'split'
1032: 'float' object has no attribute 'split'
1033: 'float' object has no attribute 'split'
1037: 'float' object has no attribute 'split'
1040: 'float' object has no attribute 'split'
1042: 'float' object has no attribute 'split'
1045: 'float' object has no attribute 'split'
1048: 'float' object has no attribute 'split'
1049: 'float' object has no attribute 'split'
1052: 'float' object has no attribute 'split'
1054: 'float' object has no attribute 'split'
1057: 'float' object has no attribute 'split'
1065: 'float' object has no attrib

In [372]:
# Convertir las fechas de las columnas "Fundación" y "Erección" de formato string a datetime

from datetime import datetime
import locale # Usar locale para configurar la lectura de las fechas en idioma español.

locale.setlocale(locale.LC_TIME, 'es_ES.UTF-8')

columnas = ['Fundación', 'Erección']

for col in columnas:
    for idx in municipios_df.index:
        try:
            municipios_df[col][idx] = datetime.strptime(municipios_df[col][idx], '%d/%B/%Y')
        except Exception as e:
            print(f"{idx}: {e}")

<ipython-input-372-f8caf235b868>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  municipios_df[col][idx] = datetime.strptime(municipios_df[col][idx], '%d/%B/%Y')


3: time data 'Entre 1627 y 1637' does not match format '%d/%B/%Y'
6: time data '1600' does not match format '%d/%B/%Y'
9: time data '1676  (345 años)' does not match format '%d/%B/%Y'
19: time data '1680/(341/años)/(primeros/asentamientos)' does not match format '%d/%B/%Y'
25: time data '1743  (278 años)' does not match format '%d/%B/%Y'
28: time data '1844' does not match format '%d/%B/%Y'
49: time data 'Anterior a la Conquista' does not match format '%d/%B/%Y'
55: time data '1907' does not match format '%d/%B/%Y'
58: time data '1521' does not match format '%d/%B/%Y'
60: time data 'Los/indígenas/Pastos,/en/época' does not match format '%d/%B/%Y'
66: time data '1536' does not match format '%d/%B/%Y'
67: time data '1840' does not match format '%d/%B/%Y'
71: time data '1815' does not match format '%d/%B/%Y'
76: time data '1723' does not match format '%d/%B/%Y'
80: strptime() argument 1 must be str, not float
81: time data '1840  \n \n180 Años' does not match format '%d/%B/%Y'
84: strptim

626: time data '1958' does not match format '%d/%B/%Y'
627: time data '1882' does not match format '%d/%B/%Y'
635: strptime() argument 1 must be str, not float
647: time data '1910' does not match format '%d/%B/%Y'
655: strptime() argument 1 must be str, not float
656: time data '1626' does not match format '%d/%B/%Y'
659: time data '1923' does not match format '%d/%B/%Y'
661: time data '1800' does not match format '%d/%B/%Y'
663: time data '1950' does not match format '%d/%B/%Y'
666: strptime() argument 1 must be str, not float
667: time data '1961' does not match format '%d/%B/%Y'
672: time data '1890  (131 años)' does not match format '%d/%B/%Y'
679: strptime() argument 1 must be str, not float
681: time data '1827' does not match format '%d/%B/%Y'
683: time data '1910' does not match format '%d/%B/%Y'
684: time data '1780' does not match format '%d/%B/%Y'
689: time data '1592   (por Melchor Guerra)' does not match format '%d/%B/%Y'
690: time data '1807' does not match format '%d/%B

105: strptime() argument 1 must be str, not float
106: time data '3 de febrero' does not match format '%d/%B/%Y'
107: time data '1814' does not match format '%d/%B/%Y'
108: strptime() argument 1 must be str, not float
110: strptime() argument 1 must be str, not float
112: strptime() argument 1 must be str, not float
113: time data '1958' does not match format '%d/%B/%Y'
115: time data '1817' does not match format '%d/%B/%Y'
116: strptime() argument 1 must be str, not float
119: strptime() argument 1 must be str, not float
120: time data '1943' does not match format '%d/%B/%Y'
125: time data '1812' does not match format '%d/%B/%Y'
128: time data '1914' does not match format '%d/%B/%Y'
130: time data '1923' does not match format '%d/%B/%Y'
131: time data 'Marzo de  1950' does not match format '%d/%B/%Y'
136: time data '1979' does not match format '%d/%B/%Y'
137: time data '1837' does not match format '%d/%B/%Y'
138: time data '1932' does not match format '%d/%B/%Y'
139: time data '1863' 

476: strptime() argument 1 must be str, not float
478: strptime() argument 1 must be str, not float
479: strptime() argument 1 must be str, not float
480: time data '1757' does not match format '%d/%B/%Y'
486: time data '1830' does not match format '%d/%B/%Y'
487: time data '1884' does not match format '%d/%B/%Y'
489: time data '1876' does not match format '%d/%B/%Y'
490: time data '1871' does not match format '%d/%B/%Y'
491: time data '1823' does not match format '%d/%B/%Y'
492: strptime() argument 1 must be str, not float
493: time data '1814' does not match format '%d/%B/%Y'
495: time data '1982' does not match format '%d/%B/%Y'
497: time data '1994' does not match format '%d/%B/%Y'
502: time data '1997' does not match format '%d/%B/%Y'
504: strptime() argument 1 must be str, not float
505: strptime() argument 1 must be str, not float
506: strptime() argument 1 must be str, not float
507: strptime() argument 1 must be str, not float
509: time data '1882' does not match format '%d/%B

821: time data '1985' does not match format '%d/%B/%Y'
824: time data '1865' does not match format '%d/%B/%Y'
826: time data '1909' does not match format '%d/%B/%Y'
832: time data '1601' does not match format '%d/%B/%Y'
834: strptime() argument 1 must be str, not float
836: strptime() argument 1 must be str, not float
839: time data '1965' does not match format '%d/%B/%Y'
843: time data '1930' does not match format '%d/%B/%Y'
845: strptime() argument 1 must be str, not float
846: time data '1777' does not match format '%d/%B/%Y'
847: time data '1776' does not match format '%d/%B/%Y'
848: time data '' does not match format '%d/%B/%Y'
849: strptime() argument 1 must be str, not float
850: strptime() argument 1 must be str, not float
853: time data '1762' does not match format '%d/%B/%Y'
855: strptime() argument 1 must be str, not float
856: time data '1776' does not match format '%d/%B/%Y'
861: time data '1595' does not match format '%d/%B/%Y'
863: strptime() argument 1 must be str, not 

In [373]:
# Para aquellas fechas incompletas extraer, por lo menos, el año.

from re import search

columnas = ['Fundación', 'Erección']

for col in columnas:
    for idx in municipios_df.index:
        try:
            if not isinstance(municipios_df[col][idx], datetime):
                municipios_df[col][idx] = search('\d{4}', municipios_df[col][idx]).group(0)
        except Exception as e:
            print(f"{idx}: {e}")


<ipython-input-373-918fd63860d4>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  municipios_df[col][idx] = search('\d{4}', municipios_df[col][idx]).group(0)


49: 'NoneType' object has no attribute 'group'
60: 'NoneType' object has no attribute 'group'
80: expected string or bytes-like object
84: expected string or bytes-like object
89: expected string or bytes-like object
142: expected string or bytes-like object
150: expected string or bytes-like object
202: 'NoneType' object has no attribute 'group'
220: 'NoneType' object has no attribute 'group'
276: expected string or bytes-like object
293: expected string or bytes-like object
312: expected string or bytes-like object
325: expected string or bytes-like object
394: expected string or bytes-like object
403: expected string or bytes-like object
414: expected string or bytes-like object
418: expected string or bytes-like object
477: expected string or bytes-like object
488: expected string or bytes-like object
515: 'NoneType' object has no attribute 'group'
522: expected string or bytes-like object
524: expected string or bytes-like object
599: expected string or bytes-like object
635: expe

1032: expected string or bytes-like object
1033: expected string or bytes-like object
1037: expected string or bytes-like object
1040: expected string or bytes-like object
1042: expected string or bytes-like object
1045: expected string or bytes-like object
1048: expected string or bytes-like object
1049: expected string or bytes-like object
1052: expected string or bytes-like object
1054: expected string or bytes-like object
1057: expected string or bytes-like object
1061: 'NoneType' object has no attribute 'group'
1065: expected string or bytes-like object
1068: expected string or bytes-like object
1070: expected string or bytes-like object
1082: expected string or bytes-like object
1084: expected string or bytes-like object
1085: expected string or bytes-like object
1091: expected string or bytes-like object
1093: expected string or bytes-like object
1096: expected string or bytes-like object
1108: expected string or bytes-like object
1109: expected string or bytes-like object
1110:

##### ** Conversión de datos númericos, modificación de strings y  manejo de valores faltantes (NaN) **

In [375]:
# Convertir valores de la columna superficie a flotantes, creando una nueva columna que muestre la unidad de medida en (km²)

municipios_df['Superficie (km²)'] = np.NaN

for idx in municipios_df.index:
    try:
        municipios_df['Superficie (km²)'][idx] = float(municipios_df['Superficie'][idx].split()[0])
    except ValueError:
        try:
            municipios_df['Superficie (km²)'][idx] = float(municipios_df['Superficie'][idx].split()[0].replace(',', '.'))
        except Exception as e:
            print(f"{idx}: {e}")

# Eliminar columna 'Superficie'
municipios_df = municipios_df.drop("Superficie", 1)

<ipython-input-375-715a539a50c3>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  municipios_df['Superficie (km²)'][idx] = float(municipios_df['Superficie'][idx].split()[0])
<ipython-input-375-715a539a50c3>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  municipios_df['Superficie (km²)'][idx] = float(municipios_df['Superficie'][idx].split()[0].replace(',', '.'))


614: could not convert string to float: '43.112.3'
815: could not convert string to float: '256M2'


In [376]:
# Convertir valores de la columna 'Altitud Media' a enteros, creando una nueva columna que muestre la unidad de medida en (msnm)

municipios_df['Altitud Media (m.s.n.m)'] = np.NaN

for idx in municipios_df.index:
    try:
        municipios_df['Altitud Media (m.s.n.m)'][idx] = float(municipios_df['Altitud Media'][idx].split()[0].replace(',', '.'))
    except Exception as e:
        print(f"{idx}: {e}")

# Eliminar columna 'Altitud Media'
municipios_df = municipios_df.drop("Altitud Media", 1)

<ipython-input-376-48130f1d33da>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  municipios_df['Altitud Media (m.s.n.m)'][idx] = float(municipios_df['Altitud Media'][idx].split()[0].replace(',', '.'))


215: could not convert string to float: '402-2987'
717: could not convert string to float: '1600/3000'
986: could not convert string to float: 'De'
1022: could not convert string to float: '85]'
1024: could not convert string to float: '2387-3200'


In [377]:
# Convertir valores de columna 'Densidad' a flotantes, creando una nueva columna que muestre la unidad de medida en (hab/km²)

municipios_df['Densidad (hab/km²)'] = np.NaN

for idx in municipios_df.index:
    try:
        municipios_df['Densidad (hab/km²)'][idx] = float(municipios_df['Densidad'][idx].split()[0])
    except (ValueError, AttributeError):
        try:
            municipios_df['Densidad (hab/km²)'][idx] = float(municipios_df['Densidad'][idx].split()[0].replace(',', '.'))
        except Exception as e:
            print(f"{idx}: {e}")


# Eliminar columna 'Densidad'
municipios_df = municipios_df.drop("Densidad", 1)

<ipython-input-377-76722e60e35f>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  municipios_df['Densidad (hab/km²)'][idx] = float(municipios_df['Densidad'][idx].split()[0].replace(',', '.'))
<ipython-input-377-76722e60e35f>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  municipios_df['Densidad (hab/km²)'][idx] = float(municipios_df['Densidad'][idx].split()[0])


75: 'float' object has no attribute 'split'
85: 'float' object has no attribute 'split'
89: 'float' object has no attribute 'split'
98: 'float' object has no attribute 'split'
103: 'float' object has no attribute 'split'
115: 'float' object has no attribute 'split'
116: 'float' object has no attribute 'split'
118: 'float' object has no attribute 'split'
119: 'float' object has no attribute 'split'
123: 'float' object has no attribute 'split'
125: 'float' object has no attribute 'split'
127: 'float' object has no attribute 'split'
128: 'float' object has no attribute 'split'
134: 'float' object has no attribute 'split'
143: 'float' object has no attribute 'split'
148: 'float' object has no attribute 'split'
150: 'float' object has no attribute 'split'
153: 'float' object has no attribute 'split'
156: 'float' object has no attribute 'split'
157: 'float' object has no attribute 'split'
158: 'float' object has no attribute 'split'
159: 'float' object has no attribute 'split'
160: 'float' o

718: 'float' object has no attribute 'split'
719: 'float' object has no attribute 'split'
725: 'float' object has no attribute 'split'
728: 'float' object has no attribute 'split'
730: 'float' object has no attribute 'split'
731: 'float' object has no attribute 'split'
733: 'float' object has no attribute 'split'
736: 'float' object has no attribute 'split'
737: 'float' object has no attribute 'split'
742: 'float' object has no attribute 'split'
747: 'float' object has no attribute 'split'
751: 'float' object has no attribute 'split'
756: 'float' object has no attribute 'split'
757: 'float' object has no attribute 'split'
759: 'float' object has no attribute 'split'
761: 'float' object has no attribute 'split'
766: 'float' object has no attribute 'split'
769: 'float' object has no attribute 'split'
771: 'float' object has no attribute 'split'
774: 'float' object has no attribute 'split'
781: 'float' object has no attribute 'split'
789: 'float' object has no attribute 'split'
790: 'floa

In [397]:
# Convertir los valores de las columnas 'Población Total' y 'Pobl. Urbana' de string a enteros

columnas = {'Población Total': 'Población Total (hab)', 'Pobl. Urbana': 'Pobl. Urbana (hab)'}

municipios_df['Pobl. Urbana (hab)'] = np.NaN
municipios_df['Población Total (hab)'] = np.NaN

for key, value in columnas.items():
    for idx in municipios_df.index:
        try:
            municipios_df[value][idx] = int(municipios_df[key][idx].split(' hab.')[0].replace(' ', '').replace(',', ''))
        except Exception as e:   
            print(f"{idx}: {e}")
    
    # Eliminar la columna correspondiente (anterior)
    municipios_df = municipios_df.drop(key, 1)


<ipython-input-397-4f322e02f4e6>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  municipios_df[value][idx] = int(municipios_df[key][idx].split(' hab.')[0].replace(' ', '').replace(',', ''))


82: invalid literal for int() with base 10: "74'500Estimado"
172: invalid literal for int() with base 10: '38838(2018)'
247: invalid literal for int() with base 10: '21716Habitantes\t10/292'
352: invalid literal for int() with base 10: '17487Habitantes(2018)'
441: invalid literal for int() with base 10: ''
990: invalid literal for int() with base 10: ''
998: 'float' object has no attribute 'split'
1006: invalid literal for int() with base 10: ''
1067: invalid literal for int() with base 10: ''
1082: invalid literal for int() with base 10: ''
1084: invalid literal for int() with base 10: ''
1095: invalid literal for int() with base 10: ''
1111: invalid literal for int() with base 10: ''
1112: invalid literal for int() with base 10: ''
13: 'float' object has no attribute 'split'
80: 'float' object has no attribute 'split'
100: 'float' object has no attribute 'split'
193: 'float' object has no attribute 'split'
269: 'float' object has no attribute 'split'
371: 'float' object has no attrib

In [432]:
# Acortar el tamaño de las coordenadas, mostrando solo la latitud y la longitud.

for idx in municipios_df.index:
    try:
        municipios_df['Coordenadas'][idx] = municipios_df['Coordenadas'][idx].split('/')[0].replace('\ufeff', '').strip()
    except Exception as e:
        print(f"{idx}: {e}")

<ipython-input-432-a195df9feb5e>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  municipios_df['Coordenadas'][idx] = municipios_df['Coordenadas'][idx].split('/')[0].replace('\ufeff', '').strip()


5: 'float' object has no attribute 'split'
20: 'float' object has no attribute 'split'


In [431]:
# Extraer solo el nombre del alcalde de cada municipio (originalmente aparece el año de posesión y el partido político)

for idx in municipios_df.index:
    try:
        municipios_df['Alcalde'][idx] = municipios_df['Alcalde'][idx].split('(')[0].strip()
    except Exception as e:
        print(f"{idx}: {e}")


<ipython-input-431-b1d4e884c15a>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  municipios_df['Alcalde'][idx] = municipios_df['Alcalde'][idx].split('(')[0].replace('\ufeff', '').strip()


93: 'float' object has no attribute 'split'
94: 'float' object has no attribute 'split'
200: 'float' object has no attribute 'split'
205: 'float' object has no attribute 'split'
224: 'float' object has no attribute 'split'
255: 'float' object has no attribute 'split'
267: 'float' object has no attribute 'split'
294: 'float' object has no attribute 'split'
327: 'float' object has no attribute 'split'
363: 'float' object has no attribute 'split'
392: 'float' object has no attribute 'split'
409: 'float' object has no attribute 'split'
417: 'float' object has no attribute 'split'
440: 'float' object has no attribute 'split'
457: 'float' object has no attribute 'split'
498: 'float' object has no attribute 'split'
501: 'float' object has no attribute 'split'
504: 'float' object has no attribute 'split'
524: 'float' object has no attribute 'split'
540: 'float' object has no attribute 'split'
553: 'float' object has no attribute 'split'
578: 'float' object has no attribute 'split'
601: 'float'

In [418]:
# En el caso de las columnas de huso horario, país y entidad los valores son iguales para todos los indices, es decir,
# Uso horario: UTC -5, Entidad: Municipio y País: Colombia. Se usa esta información para rellenar los valores nulos.

municipios_df['Huso horario'].fillna("UTC -5", inplace=True)
municipios_df['Entidad'].fillna("Municipio", inplace=True)
municipios_df['País'].fillna("Colombia", inplace=True)

In [433]:
# Cambiar el orden de las columnas dependiendo de la relevancia de la información contenida.

municipios_df = municipios_df[['Municipio', 'Departamento', 'Entidad', 'País', 'Fundación', 'Erección', 'Superficie (km²)',
                              'Altitud Media (m.s.n.m)', 'Población Total (hab)', 'Pobl. Urbana (hab)', 'Densidad (hab/km²)',
                              'Coordenadas', 'Clima', 'Gentilicio', 'Alcalde', 'Código Postal', 'Huso horario']]

##### ** ¡Trabajo finalizado! Ahora, guardar los datos en formato CSV **

In [21]:
# Guardar datos en formato CSV

municipios_df.to_csv("municipios_colombia_df.csv")